## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [11]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_0_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_exec_csv/"+'gcc6'+"_0_32_exec.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_0_32.head()

(13936744, 2)
reset_index 완료
input data shape


,bin,label
0,53,1
1,83,0
2,ec,0
3,08,0
4,e8,0


In [12]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32['label'].value_counts())

256
0    13892354
1       44390
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [13]:
######################## 
idx = gcc6_0_32[gcc6_0_32['label']==1].index  # 407, 474 ...
ls = list(idx)

# 최종 뽑을 행에 대한 index
ls_idx = [] 
left_idx, right_idx = 0, 32 # 3개씩

# 6gram
for k in range(left_idx, right_idx):
    ls_idx.extend(list(idx + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx = list(filter(lambda x: x<len(gcc6_0_32), ls_idx))
print(len(ls_idx))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub = len(ls_idx)%(right_idx)
print('나머지', sub)

ls_idx = ls_idx[:len(ls_idx)-sub]
print('최종 길이', len(ls_idx))

print('gcc6_0_32', len(ls_idx))

# loc 로 수정필요
gcc6_0_32_Ngram = gcc6_0_32.loc[ls_idx,:].copy()

1420468
나머지 20
최종 길이 1420448
gcc6_0_32 1420448


## (4) false data 만들기

In [14]:
# false data 만들기 - False 데이터 랜덤 생성
# random index
random_idx = np.random.randint(len(gcc6_0_32)-right_idx)

# 목표치
goal = len(gcc6_0_32_Ngram)/right_idx
count=0

d = pd.DataFrame(columns=gcc6_0_32.columns)

# goal 에 도달할 때까지
while count!=goal:
    
    # 진행상황 살펴보기 위함
    if count%1000==0:
        print(count, end=' ')
        
    # 랜덤 N 바이트씩 뽑음
    df = gcc6_0_32[random_idx:random_idx+right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df['label']:
        d = pd.concat([d, df])
        count+=1

print('완료')

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 완료


## (5) False Data + True Data 합치기

In [15]:
ff = pd.DataFrame(columns=gcc6_0_32.columns)

for i in range(0, int(len(d)/right_idx)):
    ff = pd.concat([ff, gcc6_0_32[i*right_idx:(i+1)*right_idx], d[i*right_idx:(i+1)*right_idx]])
    
    if i%1000==0:
        print(i, end=' ')
ff.shape

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 

(2840896, 2)

## (6) one hot encoding

In [16]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_0_32_onehot_Ngram = pd.get_dummies(ff['bin'])
gcc6_0_32_onehot_Ngram = pd.concat([ff['label'], gcc6_0_32_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(gcc6_0_32_onehot_Ngram.shape)

원핫인코딩완료
(2840896, 257)


In [17]:
# 훈련 데이터, 훈련 라벨
x_gcc6_0_32_3 = gcc6_0_32_onehot_Ngram.iloc[:,1:].to_numpy()
y_gcc6_0_32_3 = gcc6_0_32_onehot_Ngram['label'].to_numpy()
print(x_gcc6_0_32_3.shape, y_gcc6_0_32_3.shape)

x_gcc6_0_32_3 = x_gcc6_0_32_3.reshape(-1, right_idx, x_gcc6_0_32_3.shape[1])
y_gcc6_0_32_3 = y_gcc6_0_32_3.reshape(-1, right_idx, 1)

print(x_gcc6_0_32_3.shape, y_gcc6_0_32_3.shape)

(2840896, 256) (2840896,)
(88778, 32, 256) (88778, 32, 1)


## (7) 모델

In [18]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = Bidirectional(LSTM(64, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [19]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_gcc6_0_32_3, y_gcc6_0_32_3):
    model1 = Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_gcc6_0_32_3[train],
               y_gcc6_0_32_3[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_gcc6_0_32_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_0_32_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 57s 713us/step - loss: 0.0056 - accuracy: 0.9987
Epoch 2/10
79900/79900 [==============================] - 56s 706us/step - loss: 0.0025 - accuracy: 0.9992
Epoch 3/10
79900/79900 [==============================] - 56s 706us/step - loss: 0.0023 - accuracy: 0.9993
Epoch 4/10
79900/79900 [==========

accuracy_score 0.9993980907862131
recall_score 0.7683284457478006
precision_score 0.7401129943502824
f1_score 0.753956834532374
Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 57s 719us/step - loss: 0.0014 - accuracy: 0.9996
Epoch 2/10
79900/79900 [==============================] - 57s 719us/step - loss: 0.0013 - accuracy: 0.9996
Epoch 3/10
79

accuracy_score 0.999464969587745
recall_score 0.775
precision_score 0.7971428571428572
f1_score 0.7859154929577465
Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 60s 753us/step - loss: 7.6995e-04 - accuracy: 0.9998
Epoch 2/10
79900/79900 [==============================] - 60s 753us/step - loss: 6.6016e-04 - accuracy: 0.9998
Epoch 3/10
79900/7

accuracy_score 0.9996550461815724
recall_score 0.7987012987012987
precision_score 0.8723404255319149
f1_score 0.8338983050847457
Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 62s 774us/step - loss: 4.1640e-04 - accuracy: 0.9999
Epoch 2/10
79900/79900 [==============================] - 62s 770us/step - loss: 3.1025e-04 - accuracy: 0.9999
Epoc

accuracy_score 0.9997782439738679
recall_score 0.9147727272727273
precision_score 0.9070422535211268
f1_score 0.910891089108911
Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 66s 829us/step - loss: 2.7002e-04 - accuracy: 0.9999
Epoch 2/10
79900/79900 [==============================] - 64s 806us/step - loss: 1.8504e-04 - accuracy: 1.0000
Epoch

...

accuracy_score 0.9999014417661636
recall_score 0.9486486486486486
precision_score 0.975
f1_score 0.9616438356164383
Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 68s 845us/step - loss: 1.0790e-04 - accuracy: 1.0000
Epoch 2/10
79900/79900 [==============================] - 67s 833us/step - loss: 7.7699e-05 - accuracy: 1.0000
Epoch 3/10
79900/

accuracy_score 0.9999648006307726
recall_score 0.9834254143646409
precision_score 0.9888888888888889
f1_score 0.9861495844875346
Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 70s 876us/step - loss: 9.3883e-05 - accuracy: 1.0000
Epoch 2/10
79900/79900 [==============================] - 69s 860us/step - loss: 7.4987e-05 - accuracy: 1.0000
Epoc

accuracy_score 0.9999929601261546
recall_score 1.0
precision_score 0.9938271604938271
f1_score 0.9969040247678018
Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
79901/79901 [==============================] - 85s 1ms/step - loss: 5.5412e-05 - accuracy: 1.0000
Epoch 2/10
79901/79901 [==============================] - 82s 1ms/step - loss: 5.0309e-05 - accuracy: 1.0000
Epoch 3/10
79901/79901 

accuracy_score 0.99998239833277
recall_score 0.997229916897507
precision_score 0.989010989010989
f1_score 0.993103448275862
Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 128)           164352    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 32, 1)             129       
Total params: 164,481
Trainable params: 164,481
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Epoch 1/10
79901/79901 [==============================] - 82s 1ms/step - loss: 4.7308e-05 - accuracy: 1.0000
Epoch 2/10
79901/79901 [==============================] - 81s 1ms/step - loss: 3.4521e-05 - accuracy: 1.0000
Epoch 3/10
79

accuracy_score 0.9999894389996621
recall_score 0.991044776119403
precision_score 1.0
f1_score 0.9955022488755623

K-fold cross validation Accuracy: [0.9993980907862131, 0.999464969587745, 0.9996550461815724, 0.9997782439738679, 0.9998732822707818, 0.9999014417661636, 0.9999648006307726, 0.9999929601261546, 0.99998239833277, 0.9999894389996621]

K-fold cross validation Recall: [0.7683284457478006, 0.775, 0.7987012987012987, 0.9147727272727273, 0.9153005464480874, 0.9486486486486486, 0.9834254143646409, 1.0, 0.997229916897507, 0.991044776119403]

K-fold cross validation Precision: [0.7401129943502824, 0.7971428571428572, 0.8723404255319149, 0.9070422535211268, 0.9852941176470589, 0.975, 0.9888888888888889, 0.9938271604938271, 0.989010989010989, 1.0]

K-fold cross validation F1-Score: [0.753956834532374, 0.7859154929577465, 0.8338983050847457, 0.910891089108911, 0.9490084985835694, 0.9616438356164383, 0.9861495844875346, 0.9969040247678018, 0.993103448275862, 0.9955022488755623]


## (9) 평가지표

In [20]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9998000672655702
10-Fold Cross_validation. Recall : 0.9092451774200114
10-Fold Cross_validation. Precision : 0.9248659686586945
10-Fold Cross_validation. F1-Score : 0.9166973362290545
